In [26]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from geneticalgorithm import geneticalgorithm as ga

In [27]:
call_df = pd.read_csv('') # Call options
History = pd.read_csv('') # Stock-price

In [28]:
call_df = call_df.set_index('Date')

In [29]:
call_df.head(5)

,stike,Close,nDiff,r,Closing_price,sigma_20
Date,,,,,,
1/1/2018,9400.0,2397.15,724.0,7.33,10486.4502,0.0059
1/1/2018,11000.0,40.60,52.0,7.33,10486.4502,0.0059
1/1/2018,11050.0,32.25,52.0,7.33,10486.4502,0.0059
1/1/2018,4100.0,3060.65,178.0,7.33,10486.4502,0.0059
1/1/2018,4000.0,6262.10,178.0,7.33,10486.4502,0.0059


The volatility is already calculated as the rolling average of the underlying asset, with a window size equal to 20. This is one of the most, if not the most, comon way of estimating volatility.

In [30]:
def black_scholes(row):
    S = row.Closing_price
    X = row.stike
    T = row.nDiff / 365
    r = row.r / 100
    σ = row.sigma_20
    d1 = (np.log(S / X) + (r + (σ ** 2) / 2) * T) / (σ * (T ** .5))
    d2 = d1 - σ * (T ** .5)
    C = S * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    return C

In [ ]:
call_df['black_scholes_pred'] = call_df.apply(black_scholes, axis=1)

In [34]:
def gen_normal(self,n_step=25):
        x0=0
        w = np.ones(n_step)*x0
        
        for i in range(1,n_step):
            # Sampling from the Normal distribution
            yi = np.random.normal()
            # Weiner process
            w[i] = w[i-1]+(yi/np.sqrt(n_step))
        
        return w

In [ ]:
sigma, mu = BS_params
n_step = 25 #int(deltaT/dt)
time_vector = np.linspace(0,1,num=n_step)# Stock variation
stock_var = (mu-(sigma**2/2))*time_vector
# Forcefully set the initial value to zero for the stock price simulation
x0=0
# Weiner process (calls the `gen_normal` method)
weiner_process = sigma*gen_normal(n_step)
# Add two time series, take exponent, and multiply by the initial stock price
s = History[0].close*(np.exp(stock_var+weiner_process))
s.to_csv('./BS-stock')

# GA Calibration

In [35]:
varbound=np.array([[0,1]]*2) #Variable Boundries
algorithm_param = {'max_num_iteration': 3000,\
                   'population_size':100,\
                   'mutation_probability':0.1,\
                   'elit_ratio': 0.01,\
                   'crossover_probability': 0.5,\
                   'parents_portion': 0.3,\
                   'crossover_type':'uniform',\
                   'max_iteration_without_improv':30}

In [36]:
def GA_BS(Y):
    def f(X):
        sigma,mu=X
        n_step = 25 #int(deltaT/dt)
        time_vector = np.linspace(0,1,num=n_step)# Stock variation
        stock_var = (mu-(sigma**2/2))*time_vector
        # Forcefully set the initial value to zero for the stock price simulation
        x0=0
        # Weiner process (calls the `gen_normal` method)
        weiner_process = sigma*gen_normal(n_step)
        # Add two time series, take exponent, and multiply by the initial stock price
        s = Y[0]*(np.exp(stock_var+weiner_process))
        s.to_csv('./BS-stock-GA')
        return np.mean((s-Y)**2)
    model=ga(function=f,\
            dimension=2,\
            variable_type='real',\
            variable_boundaries=varbound,\
            algorithm_parameters=algorithm_param,
         convergence_curve=False,
         progress_bar=False)

    model.run()
    return model.best_variable[0]

In [ ]:
call_df[['sigma']] = History.Close.apply(GA_BS, raw=True)

In [43]:
History_sigma=History.drop(['Close'],axis=1)
options_df_with_sigma = call_df.join(History_sigma.set_index('Date'))
call_df=options_df_with_sigma.dropna()

In [45]:
def black_scholes_GA(row):
    S = row.Closing_price
    X = row.stike
    T = row.nDiff / 365
    r = row.r / 100
    σ = row.sigma
    d1 = (np.log(S / X) + (r + (σ ** 2) / 2) * T) / (σ * (T ** .5))
    d2 = d1 - σ * (T ** .5)
    C = S * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    return C

In [ ]:
call_df['black_scholes_pred-GA'] = call_df.apply(black_scholes_GA, axis=1)

# Save dataframe

In [ ]:
call_df.to_csv('./BS-input-output')

# Metrics

In [ ]:
from .utilties import utilties

In [55]:
line1 = utilties.error_metrics(call_df['Close'], call_df['black_scholes_pred'])

In [56]:
line2 = utilties.error_metrics(call_df['Close'], call_df['black_scholes_pred-GA'])

In [ ]:
for line in ([*line1], [*line2]):
  print('& {:.2f} & {:.2f}% & {:.2f}% & {:.2f}% & {:.2f}% & {:.2f}% & {:.2f}% \\\\'.format(*line))